In [2]:
import numpy as np
import pandas as pd
from collections import Counter
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN

In [3]:
import pandas as pd
import logging

# Log fayl manzili
log_path = r"C:\Users\Rasulbek907\Desktop\Project_MP\Log\data_loader.log"

# Log sozlamalari
logging.basicConfig(
    filename=log_path,
    filemode='a',  # Append mode
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

try:
    logging.info("CSV fayl o'qilmoqda:...")
    df = pd.read_csv(r"C:\Users\Rasulbek907\Desktop\Project_MP\Data\Feature_Selection\Filtered_Features.csv")
    logging.info(f"Fayl muvaffaqiyatli o'qildi. Satırlar soni: {len(df)} ustunlar soni: {len(df.columns)}")
except Exception as e:
    logging.error(f"CSV faylni o'qishda xatolik: {e}")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4993 entries, 0 to 4992
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0.1           4993 non-null   int64  
 1   Unnamed: 0             4993 non-null   int64  
 2   name_length            4993 non-null   float64
 3   is_organic             4993 non-null   float64
 4   quantity_value         4993 non-null   float64
 5   quantity_unit_encoded  4993 non-null   float64
 6   category_depth         4993 non-null   float64
 7   country_count          4993 non-null   float64
 8   product_age_days       4993 non-null   float64
 9   created_month          4993 non-null   float64
 10  main_category_encoded  4993 non-null   float64
 11  nova_group             4993 non-null   float64
dtypes: float64(10), int64(2)
memory usage: 468.2 KB


In [5]:
df.head()

,Unnamed: 0.1,Unnamed: 0,name_length,is_organic,quantity_value,quantity_unit_encoded,category_depth,country_count,product_age_days,created_month,main_category_encoded,nova_group
0,0,0,-0.691811,-0.121809,-1.138126,-1.013055,-0.069824,-0.588663,0.954730,0.590746,-1.202998,5.0
1,1,1,-1.162028,-0.121809,-0.846880,-0.425509,-1.835039,-0.180838,0.674675,0.890838,1.126987,3.0
2,2,2,-0.691811,-0.121809,-1.272881,0.833518,-0.364026,-0.588663,0.077223,0.890838,-1.202998,1.0
3,3,3,-0.221594,-0.121809,-1.277228,0.833518,-0.069824,-0.588663,1.276997,0.890838,-1.202998,1.0
4,4,4,-1.162028,-0.121809,-1.138126,-1.013055,-0.364026,-0.588663,-0.105422,-0.909710,-1.116914,5.0


In [8]:
df["nova_group"].unique()

array([5., 3., 1., 4., 2.])

In [9]:
df["nova_group"] = df["nova_group"].astype(int)

In [11]:
X = df.drop('nova_group', axis=1)
y = df['nova_group']

In [12]:
print("Original class distribution:", Counter(y))

Original class distribution: Counter({4: 2624, 3: 1099, 1: 567, 5: 491, 2: 212})


# 🧮 Tahlil

# Bu taqsimotdan ko‘rinadiki, ma’lumotlar balanssiz (imbalanced).
# Eng katta sinf — 4 (2624 ta, ya’ni ~52%)
# Eng kichik sinf — 2 (212 ta, ya’ni ~4%)
# 1, 3, 5 sinflar ham nisbatan kam.

# SMOTE

In [14]:
smote = SMOTE(random_state=42,k_neighbors=1)
X_smote, y_smote = smote.fit_resample(X, y)

In [16]:
print("SMOTE class distribution:", Counter(y_smote))
print("SMOTE X shape:", X_smote.shape)

SMOTE class distribution: Counter({5: 2624, 3: 2624, 1: 2624, 4: 2624, 2: 2624})
SMOTE X shape: (13120, 11)


In [17]:
X_smote, y_smote

(       Unnamed: 0.1  Unnamed: 0  name_length  is_organic  quantity_value  \
 0                 0           0    -0.691811   -0.121809       -1.138126   
 1                 1           1    -1.162028   -0.121809       -0.846880   
 2                 2           2    -0.691811   -0.121809       -1.272881   
 3                 3           3    -0.221594   -0.121809       -1.277228   
 4                 4           4    -1.162028   -0.121809       -1.138126   
 ...             ...         ...          ...         ...             ...   
 13115          4838        4838    -0.232006   -0.121809        0.230232   
 13116           141         141    -0.717620   -0.121809       -0.584608   
 13117          1108        1108    -1.162028   -0.121809       -0.836134   
 13118          1728        1728    -1.162028   -0.121809        0.891899   
 13119          4241        4241    -0.723256   -0.121809        0.891899   
 
        quantity_unit_encoded  category_depth  country_count  product_age_

In [18]:
import pandas as pd
import os

# 🧩 X va y ni bitta DataFrame'ga birlashtirish
smote_df = pd.concat([pd.DataFrame(X_smote, columns=X.columns),
                      pd.DataFrame(y_smote, columns=["nova_group"])],
                     axis=1)

# 📁 Saqlash manzili
save_path = r"C:\Users\Rasulbek907\Desktop\Project_MP\Data\Sampling_Data"
os.makedirs(save_path, exist_ok=True)

# 📄 CSV faylga saqlash
file_path = os.path.join(save_path, "Sampling_SMOTE.csv")
smote_df.to_csv(file_path, index=False)

print(f"✅ SMOTE balanslangan ma’lumot saqlandi:\n📂 {file_path}")
print("📊 Yangi DataFrame shakli:", smote_df.shape)

✅ SMOTE balanslangan ma’lumot saqlandi:
📂 C:\Users\Rasulbek907\Desktop\Project_MP\Data\Sampling_Data\Sampling_SMOTE.csv
📊 Yangi DataFrame shakli: (13120, 12)
